# Analytics Vidhya

## Janatahack: Machine Learning in Agriculture

https://datahack.analyticsvidhya.com/contest/janatahack-machine-learning-in-agriculture/#ProblemStatement


### Import necessary libraries and adjust the settings

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import StratifiedKFold,KFold
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

%matplotlib inline

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

pd.options.display.max_columns = None
pd.options.display.max_rows = None

pd.set_option('display.max_colwidth', -1)

### Read data files, get basic stats and overview of it

In [ ]:
train = pd.read_csv('data/train_yaOffsB.csv',index_col=None,sep=',')
print(train.shape)
print(train.isnull().sum())
train.head(2)

In [ ]:
test = pd.read_csv('data/test_pFkWwen.csv')
print(test.shape)
print(test.isnull().sum())
test.head(2)

In [ ]:
train.dtypes

In [ ]:
test.dtypes

In [ ]:
train.head(2)

### Get rid of NULLs

In [ ]:
train.Number_Weeks_Used.fillna(-99,inplace=True)
test.Number_Weeks_Used.fillna(-99,inplace=True)

### Setting up folds

In [ ]:
K = 5
kf = KFold(n_splits = K, random_state = 7, shuffle = True)
skf = StratifiedKFold(n_splits = K, random_state = 7, shuffle = True)

In [ ]:
MAX_ROUNDS = 1000
OPTIMIZE_ROUNDS = False

In [ ]:
X = train.drop(columns=['ID','Crop_Damage'],axis=1)
y = train['Crop_Damage']
X_test = test.drop(columns='ID',axis=1)
y_valid_pred = 0*y
y_test_pred = 0
accuracy = 0
result={}
cat_columns = []
j=1
model = CatBoostClassifier(n_estimators=MAX_ROUNDS,verbose=False)
for i, (train_index, test_index) in enumerate(kf.split(train)):
    y_train, y_valid = y.iloc[train_index], y.iloc[test_index]
    X_train, X_valid = X.iloc[train_index,:], X.iloc[test_index,:]
    print("\nFold - ", j)
    
    if OPTIMIZE_ROUNDS:
        fit_model = model.fit( X_train, y_train, 
                               eval_set=[X_valid, y_valid],cat_features=cat_columns,
                               use_best_model=True
                             )
        print( "N trees = ", model.tree_count_ )
    else:
        fit_model = model.fit( X_train, y_train,cat_features=cat_columns )
        
    pred = fit_model.predict(X_valid)
    y_valid_pred.iloc[test_index] = pred.reshape(-1)
    print(accuracy_score(y_valid,pred))
    accuracy+=accuracy_score(y_valid,pred)
    y_test_pred += fit_model.predict(X_test)
    result[j]=fit_model.predict(X_test)
    j+=1
results = y_test_pred / K  # Average test set predictions
print(accuracy/5)

In [ ]:
red = pd.DataFrame()
for i in range(1, 6):
    red = pd.concat([red,pd.DataFrame(result[i])],axis=1)
red.columns=['1','2','3','4','5']

In [ ]:
final = pd.DataFrame(data=test['ID'], index=None)

In [ ]:
final['Crop_Damage'] = red.mode(axis=1)[0]

In [ ]:
final.head()

In [ ]:
final.to_csv('base_test_catboost.csv',index = False)